In [46]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from itertools import combinations
from sklearn.metrics import explained_variance_score, r2_score

from sklearn import linear_model
reg = linear_model.LinearRegression()
reg1 = linear_model.BayesianRidge()
reg2 = linear_model.RANSACRegressor()
reg3 = linear_model.LassoLars(alpha=.1)
reg4 = linear_model.RidgeCV(alphas=[0.1, 1.0, 10.0])
reg5 = linear_model.ElasticNetCV()
reg6 = linear_model.SGDRegressor()
reg7 = linear_model.PassiveAggressiveRegressor()
reg8 = linear_model.TheilSenRegressor()
reg9 = linear_model.HuberRegressor()
from sklearn import svm
clf = svm.SVR(C=1.0, epsilon=0.2)
models = [clf, reg, reg1,reg3,reg5]

models_desc = [x.__class__.__name__ for x in models]

# Getting the Data

In [22]:
df = pd.read_csv('../datasets/UnnormalizedCrimeData.csv');
df = df.replace('?',np.NAN)

goal_features = ['murders', 'murdPerPop', 'rapes', 'rapesPerPop', 'robberies','robbbPerPop',
                 'assaults', 'assaultPerPop', 'burglaries', 'burglPerPop', 'larcenies', 'larcPerPop',
                 'autoTheft', 'autoTheftPerPop', 'arsons', 'arsonsPerPop', 'violentPerPop', 'nonViolPerPop']

non_predictive_features = ['communityname', 'state', 'countyCode', 'communityCode', 'fold']

features = [x for x in df.columns if x not in goal_features and x not in non_predictive_features]
len(features)

124

In [23]:
def drop_rows_with_null_goal_feature(old_df, feature):
    new_df = old_df.dropna(subset=[feature])
    return new_df

In [49]:
x_count = 0

r2_scores =[]
for goal_feature in goal_features:
    r2_scores.append([])
    goal_df = drop_rows_with_null_goal_feature(df, goal_feature)
    print goal_feature
    print goal_df[goal_feature].describe()
    print "\n\n"
    
    goal_df[[goal_feature]] = goal_df[[goal_feature]].apply(pd.to_numeric)
    imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imr = imr.fit(goal_df[features])
    imputed_data = imr.transform(goal_df[features]);

    df_X_train, df_X_test, df_y_train, df_y_test = \
            train_test_split(imputed_data, goal_df[goal_feature], test_size=0.2)
    y_count = 0
    for model in models:
        model.fit(df_X_train, df_y_train)
        r2_scores[-1].append(r2_score(df_y_test, model.predict(df_X_test)))
    print r2_scores
    
    xxx = [ct+1 for ct in range(len(models))]
    plt.plot(xxx, r2_scores[-1],label = goal_feature)
    #plt.plot(models_desc,r2_scores[-1])
    plt.xticks(xxx, models_desc, rotation='vertical')
    plt.legend()
    
    
    

murders
count    2215.000000
mean        7.764786
std        58.166468
min         0.000000
25%         0.000000
50%         1.000000
75%         3.000000
max      1946.000000
Name: murders, dtype: float64



[[-0.035911732187266399, 0.77436163476955366, 0.7877456342894581, 0.7815355744381054, 0.2170684872734886]]
murdPerPop
count    2215.000000
mean        5.859296
std         9.156829
min         0.000000
25%         0.000000
50%         2.170000
75%         8.365000
max        91.090000
Name: murdPerPop, dtype: float64



[[-0.035911732187266399, 0.77436163476955366, 0.7877456342894581, 0.7815355744381054, 0.2170684872734886], [-0.14312161968855053, 0.38953804585915586, 0.49809860060287536, 0.26868053477202269, -0.13979413022529763]]
rapes
count     2007
unique     171
top          0
freq       189
Name: rapes, dtype: object



[[-0.035911732187266399, 0.77436163476955366, 0.7877456342894581, 0.7815355744381054, 0.2170684872734886], [-0.14312161968855053, 0.38953804585915586, 0.4980

In [50]:
plt.show()

In [56]:
bayesian_score = [r2_scores[ip][2] for ip in range(len(r2_scores))]

arr = np.array(bayesian_score)
order = arr.argsort()
print order
ranks = order.argsort()
print ranks



[15  3 11 17  7  1 14 16 13  9  5  6  0  2 12 10  8  4]
[12  5 13  1 17 10 11  4 16  9 15  2 14  8  6  0  7  3]


In [57]:
for i in range(len(ranks)):
    prin[i]

0.787745634289
0.498098600603
0.803493025491
0.342324384428
0.978664389066
0.73890695299
0.74791752904
0.429925899815
0.933799630769
0.618929639164
0.913497730856
0.377950695291
0.83483771429
0.600818084465
0.538061881726
0.174178787316
0.585104548185
0.414024991182
